In [ ]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from urllib.parse import quote
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyquery import PyQuery as pq

browser = webdriver.Firefox()
wait = WebDriverWait(browser, 10)
KEYWORD = "iPad"
MAX_PAGE = 3

def index_page(page):
    """
    抓取索引页
    :param page:页码 
    """
    print("正在抓取第{0}页".format(page))
    try:
        url = "https://s.taobao.com/search?q=" + quote(KEYWORD)
        browser.get(url)
        if page > 1:
            input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,
                                                                    "#mainsrp-pager > div > div > div > div.form > input")))
            submit = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                                            "#mainsrp-pager > div > div > div > div.form > span.btn.J_Submit")))
            input.clear()
            input.send_keys(page)
            submit.click()
        wait.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR,
                                                     "#mainsrp-pager > div > div > div > ul > li.item.active > span"),str(page)))
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#mainsrp-itemlist .items .item")))
        get_products()
    except TimeoutError:
        index_page(page)

def get_products():
    """
    提取商品数据
    """
    html = browser.page_source
    doc = pq(html)
    items = doc("#mainsrp-itemlist .items .item").items()
    for item in items:
        product = {
            "image": item.find(".pic .img").attr("data-src"),
            "price": item.find(".price").text(),
            "deal": item.find(".deal-cnt").text(),
            "title": item.find(".title").text(),
            "shop": item.find(".shop").text(),
            "location": item.find(".location").text()
        }
        print(product)

def main():
    """
    遍历每一页
    """
    for i in range(1, MAX_PAGE + 1):
        index_page(i)
    browser.close()

In [ ]:
main()